In [2]:
%pip install tweepy==3.10.0

import json
import tweepy

Note: you may need to restart the kernel to use updated packages.


In [3]:
with open('./credential.json','r') as f:
    credential = json.load(f)

CONSUMER_KEY = credential['twitter_api_key']
CONSUMER_SECRET = credential['twitter_api_secret_key']
ACCESS_TOKEN = credential['twitter_access_token']
ACCESS_TOKEN_SECRET = credential['twitter_access_token_secret']
bearer_token = credential['bearer_token']

In [8]:
class SimpleStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print(status)
        #tweet = json.dumps({'id': status.id, 'created_at': status.created_at, 'text': status.text}, default=str)
    def on_error(self, status_code):
        print(status_code)
        if status_code == 420:
            return False

stream_listener = SimpleStreamListener()
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
twitterStream = tweepy.Stream(auth, stream_listener)
twitterStream.filter(track=['apple', 'samsung'])

Status(_api=<tweepy.api.API object at 0x7f0618d80c10>, _json={'created_at': 'Mon May 09 15:15:51 +0000 2022', 'id': 1523683172738732034, 'id_str': '1523683172738732034', 'text': 'RT @eth_apple: 🍎APPLE X TrippinApe🍎\n\nWL X 3\n\nSolana Tier 1 Project\nTrippinApe with perfect art and community\n\n1️⃣ Follow \n@TrippinApeNFT\n@e…', 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1509882662466437125, 'id_str': '1509882662466437125', 'name': 'Laurel Tennyson', 'screen_name': 'LisaWil65490883', 'location': 'Fort Myers', 'url': None, 'description': 'Palestinian-American #NFT #Artist, #poet, #lawyer, #woman, #activist In USA', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 6, 'friends_count': 48, 'listed_count': 0, 'favourites_

KeyboardInterrupt: 

In [ ]:
print(tweepy.__version__)